In [1]:
!pip install groq
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [2]:
import time
from datasets import load_dataset
import os
from groq import Groq

In [3]:
ds = load_dataset("ChilleD/SVAMP")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/675 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/111k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/54.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/700 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

Zero_Shot

In [5]:
client = Groq(api_key='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')
#'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'
#'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'

In [7]:
student_system = {"role": "system",
                  "content": "Never forget you are a brilliant intelligent STUDENT who solves complex math problems with help of TEACHER guidance. \
                  YOUR TASK: Together with your TEACHER's help find correct answer to math questions. The QUESTION is VERIFIED and is ALWAYS CORRECT. Never forget your TASK. \
                  You as a STUDENT will get a math question from USER and in return you will write, detailed explaination to answer that question. \
                  Your TEACHER will provide you with feedback based upon what you did. \
                  If it is incorrect at some part you will be told what you did wrong and so you will modify your answer based upon the TEACHER's feedback. \
                  If it is correct, you will not modify your answer and return the same answer just by checking your calculations. \
                  Always follow the given pattern TEMPLATE to answer. \
                  TEMPLATE: \
                  Explanation Body Paragraph\
                  **Final Answer: <SINGLE NUMERICAL ANSWER>** \
                  Make sure the last line of any response is ALWAYS - **Final Answer: <SINGLE NUMERICAL ANSWER>** "}

teacher_system = {"role": "system",
                  "content": "Never forget you are an intelligent helpful TEACHER who helps his STUDENT to solve complex math problems. \
                  You will have a question and the answer that student writes. The given QUESTION is VERIFIED and is ALWAYS CORRECT. STICK TO THE GIVEN QUESTION \
                  YOUR TASK: Guide and help your STUDENT to find correct answer and explaination to math questions. Never forget your TASK. \
                  You as a TEACHER will check if the explaination of the answer provided by the STUDENT is correct or not and provide him feedback. \
                  If the explaination is incorrect you will tell which part of the answer did the STUDENT went wrong, whether it is calculation or the approach the student used to get the answer \
                  If the explaination is correct you will praise the STUDENT and tell him to write the answer again. \
                  Also instruct the student every time to follow the given template \
                  TEMPLATE: \
                  Explanation Body Paragraph\
                  **Final Answer: <SINGLE NUMERICAL ANSWER>** \
                  Make sure the last line of any response is ALWAYS - **Final Answer: <SINGLE NUMERICAL ANSWER>** "}



In [8]:
# model_name = 'mixtral-8x7b-32768'
def multi_agent_chat(query):
    student_chat = []
    teacher_chat = []
    student_chat.append(student_system)
    teacher_chat.append(teacher_system)
    for i in range(2):
        # print('___________________________________________')
        student_chat.append({"role": "user", "content": query})
        teacher_chat.append({"role": "user", "content": query})
        student_response = client.chat.completions.create(model='gemma2-9b-it', messages=student_chat, temperature=0)
        # print(student_response.choices[0].message.content)
        teacher_chat.append({"role": "user", "content": student_response.choices[0].message.content})
        teacher_response = client.chat.completions.create(model='llama-3.1-8b-instant', messages=teacher_chat, temperature=0, max_tokens=2048)
        # print(teacher_response.choices[0].message.content)
        student_chat.append({"role": "user", "content": teacher_response.choices[0].message.content})
        student_response = client.chat.completions.create(model='gemma2-9b-it', messages=student_chat, temperature=0, max_tokens=2048)
        # print('___________________________________________')
    # print("FINAL RESPONSE")
    # print(student_response.choices[0].message.content)
    return student_response.choices[0].message.content

In [9]:
import re
import time
global_counter = 0

def ans_extractor(output):
    global global_counter
    matcha = re.search(r'Answer:\s*([0-9]+)', output)
    if matcha:
        final_answer = matcha.group(1)
        return final_answer
    else:
        global_counter += 1
        return 0

In [10]:
pt = []
gt = []
counter=0
for i in ds['test']:
    counter+=1
    print(counter, global_counter)
    query = i['question_concat']
    explaination = multi_agent_chat(query)
    answer = ans_extractor(explaination)
    if answer!=None:
        pt.append(answer)
        gt.append(i['Answer'])

1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 1
10 1
11 1
12 1
13 1
14 1
15 1
16 1
17 2
18 2
19 3
20 3
21 3
22 3
23 3
24 3
25 3
26 4
27 5
28 5
29 5
30 5
31 5
32 6
33 6
34 6
35 6
36 6
37 6
38 6
39 6
40 6
41 6
42 6
43 6
44 6
45 7
46 7
47 7
48 7
49 7
50 7
51 7
52 7
53 7
54 8
55 8
56 8
57 9
58 9
59 9
60 9
61 9
62 10
63 10
64 10
65 10
66 10
67 11
68 11
69 11
70 11
71 11
72 11
73 11
74 11
75 11
76 11
77 12
78 12
79 12
80 12
81 12
82 13
83 13
84 13
85 13
86 13
87 13
88 14
89 14
90 15
91 15
92 15
93 16
94 16
95 16
96 16
97 16
98 16
99 16
100 16
101 16
102 16
103 16
104 16
105 16
106 16
107 16
108 16
109 17
110 17
111 17
112 17
113 17
114 17
115 17
116 17
117 17
118 17
119 17
120 17
121 17
122 17
123 17
124 17
125 17
126 17
127 17
128 17
129 17
130 17
131 17
132 18
133 19
134 20
135 20
136 20
137 20
138 20
139 20
140 20
141 20
142 20
143 20
144 20
145 20
146 20
147 20
148 21
149 21
150 21
151 21
152 21
153 21
154 21
155 22
156 22
157 22
158 23
159 24
160 25
161 25
162 25
163 25
164 25
165 25
166 25
167 25


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `gemma2-9b-it` in organization `org_01j8za9skrfw88xyrext8w5nza` on : Limit 500000, Used 500055, Requested 355. Please try again in 1m10.966799999s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

In [11]:
from sklearn.metrics import accuracy_score
print(global_counter)
accuracy = accuracy_score(gt, pt)
print(f'Accuracy: {accuracy}')

29
Accuracy: 0.7567567567567568
